In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import pandas_ta as ta
import numpy as np
import code

In [46]:
# load data and add basic modifications
tsla_backtest = pd.read_csv('/Users/kylekent/Library/CloudStorage/Dropbox/divinity/output/backtestv0.0.2{}2016-01-012022-01-28.csv'.format('TSLA'))
tsla_backtest.columns.values[0] = 'indexValue'
tsla_backtest['ds'] = pd.to_datetime(tsla_backtest['ds'])
tsla_backtest = tsla_backtest.reindex(index=tsla_backtest.index[::-1])
tsla_backtest = tsla_backtest.reset_index(drop = True)
tsla_backtest.head()

,indexValue,ds,yhat,yhat_lower,yhat_upper,y
0,253,2017-01-04,38.591362,34.443162,43.111233,45.397999
1,253,2017-01-05,38.896702,34.625595,43.032766,45.349998
2,253,2017-01-06,39.377669,35.202265,43.426187,45.801998
3,251,2017-01-09,40.235927,36.469868,44.106088,46.256001
4,252,2017-01-10,40.371809,36.251111,44.817703,45.973999


In [52]:
# add functions to process backtest info
def edge_intersection(x1: int, y1: int, x2: int, y2: int, x3: int, y3: int, x4: int, y4: int) -> list:
    """Intersection point of two line segments in 2 dimensions

    params:
    ----------
    x1, y1, x2, y2 -> coordinates of line a, p1 ->(x1, y1), p2 ->(x2, y2), 

    x3, y3, x4, y4 -> coordinates of line b, p3 ->(x3, y3), p4 ->(x4, y4)

    Return:
    ----------
    list
        A list contains x and y coordinates of the intersection point,
        but return an empty list if no intersection point.

    """

    # None of lines' length could be 0.
    ((x1 == x2) & (y1 == y2)) | ((x3 == x4) & (y3 == y4))
    if (((x1 == x2) & (y1 == y2)) | ((x3 == x4) & (y3 == y4))):
        #print('line length 0')
        return []

    # The denominators for the equations for ua and ub are the same.
    den = ((y4 - y3) * (x2 - x1) - (x4 - x3) * (y2 - y1))
    (y4 - y3) * (x2 - x1)
    (x4 - x3) * (y2 - y1)

    # Lines are parallel when denominator equals to 0,
    # No intersection point
    if den == 0:
        #print('parallel')
        return []

    # Avoid the divide overflow
    ua = ((x4 - x3) * (y1 - y3) - (y4 - y3) * (x1 - x3)) / (den + 1e-16)
    ub = ((x2 - x1) * (y1 - y3) - (y2 - y1) * (x1 - x3)) / (den + 1e-16)

    # if ua and ub lie between 0 and 1.
    # Whichever one lies within that range then the corresponding line segment contains the intersection point.
    # If both lie within the range of 0 to 1 then the intersection point is within both line segments.
    if (ua < 0 or ua > 1 or ub < 0 or ub > 1):
        #print('ua ub lie between 0 and 1')
        return []

    # Return a list with the x and y coordinates of the intersection
    x = x1 + ua * (x2 - x1)
    y = y1 + ua * (y2 - y1)
    return [x, y]

def prophetCross(data, fast, slow, args, MA = 'simple'):
    # what type of moving average
    if MA == 'simple':
        data['maFast'] = ta.sma(data['y'], length = fast)
        data['maSlow'] = ta.sma(data['y'], length = slow)
    if MA == 'exponential':
        data['maFast'] = ta.ema(data['y'], length = fast)
        data['maSlow'] = ta.ema(data['y'], length = slow)
    # convert ds to unix timestamp for next step
    #data['ds'] = pd.to_datetime(data['ds']).astype(int)/10**9
    dataDateModified = data.copy()
    dataDateModified['ds'] = pd.to_datetime(data['ds']).astype(int)/10**9
    # get intersections of points
    crossList = []
    for i in range(1, len(data)-1):
        crossList.append(edge_intersection(dataDateModified['ds'][i], \
                                           dataDateModified['maFast'][i], \
                                           dataDateModified['ds'][i+1], \
                                           dataDateModified['maFast'][i+1], \
                                           dataDateModified['ds'][i], \
                                           dataDateModified['maSlow'][i], dataDateModified['ds'][i+1], \
                                           dataDateModified['maSlow'][i+1]))
    crossDf = pd.DataFrame(crossList)
    print(crossDf)
    # name crossDf 
    crossDf.columns = ['crossDate', 'crossY']
    # add cross data to df
    #data['crossDate'] = crossDf.iloc[:,0]
    #data['crossY'] = crossDf.iloc[:,1]
    
    # convert unix timestamps back to datetime
    #data['ds'] = pd.to_datetime(data['ds'], unit = 's')
    crossDf['crossDate'] = pd.to_datetime(crossDf['crossDate'], unit = 's')
    
    # signal logic
    #crossDf['sell'] = data['yhat_upper'].reset_index(drop=True) < crossDf['crossY'].reset_index(drop=True)
    #crossDf['buy'] = data['yhat_lower'].reset_index(drop=True) > crossDf['crossY'].reset_index(drop=True)
    
    # return df
    return crossDf
    

def performance(data, start_value = 1000):
    # change in stock value
    data['percentChange'] = data['y']/data['y'].shift(1)
    # benchmarking
    data.loc[0, 'benchmarkValue'] = start_value
    for i in range(1, len(data)):
        data['benchmarkValue'][i] = data['benchmarkValue'][i-1]*data['percentChange'][i]
    # logic for position of strategy
    data.loc[0, 'position'] = 'out'
    data.loc[data['buy'] == True, 'position'] = 'in'
    data.loc[data['sell'] == True, 'position'] = 'out'
    data['position'] = data['position'].ffill()
    data.loc[0, 'stratValue'] = start_value
    for i in range(1, len(data)):
        if data['position'][i] == 'in':
            data['stratValue'][i] = data['stratValue'][i-1]*data['percentChange'][i]
        else:
            data['stratValue'][i] = data['stratValue'][i-1]
    # count number of changes
    data.loc[0, 'grp'] = 1
    for i in range(1, len(data)):
        if data['position'][i] != data['position'][i-1]:
            data.loc[i, 'grp'] = 1
        else:
            data.loc[i, 'grp'] = data.loc[i-1, 'grp'] + 1
    grpOneDf = data[data['grp'] == 1]
    grpOneDf['profit'] = grpOneDf['stratValue'] - grpOneDf['stratValue'].shift(1)
    grpOneDf.loc[grpOneDf['position'] == 'in', 'profit'] = pd.NA
    # put grpOneDf profit back into data
    profitIndex = grpOneDf[grpOneDf['profit'].notna()].index
    data.loc[profitIndex, 'profit'] = grpOneDf.loc[profitIndex, 'profit']


In [55]:
# add new columns to backfit data
xx = prophetCross(tsla_backtest, fast = 3, slow = 5, args = None, MA = 'simple')
print(xx)

                 0          1
0              NaN        NaN
1              NaN        NaN
2              NaN        NaN
3              NaN        NaN
4     1.484167e+09  45.962531
...            ...        ...
1269           NaN        NaN
1270           NaN        NaN
1271           NaN        NaN
1272           NaN        NaN
1273           NaN        NaN

[1274 rows x 2 columns]
                         crossDate     crossY
0                              NaT        NaN
1                              NaT        NaN
2                              NaT        NaN
3                              NaT        NaN
4    2017-01-11 20:28:42.573531648  45.962531
...                            ...        ...
1269                           NaT        NaN
1270                           NaT        NaN
1271                           NaT        NaN
1272                           NaT        NaN
1273                           NaT        NaN

[1274 rows x 2 columns]


/var/folders/kc/dkd5zshj10bczq4svz0q8v180000gn/T/ipykernel_19184/1633082569.py:63: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  dataDateModified['ds'] = pd.to_datetime(data['ds']).astype(int)/10**9


In [22]:
tsla_backtest.head()

,indexValue,ds,yhat,yhat_lower,yhat_upper,y,maFast,maSlow,crossDate,crossY,sell,buy,percentChange,benchmarkValue,position,stratValue,grp,profit
0,253,2017-01-04,38.591362,34.443162,43.111233,45.397999,NaN,NaN,NaT,NaN,False,False,NaN,1000.000000,in,1000.000000,1.0,NaN
1,253,2017-01-05,38.896702,34.625595,43.032766,45.349998,NaN,NaN,NaT,NaN,False,False,0.998943,998.942677,in,998.942677,2.0,NaN
2,253,2017-01-06,39.377669,35.202265,43.426187,45.801998,45.516665,NaN,NaT,NaN,False,False,1.009967,1008.899056,in,1008.899056,3.0,NaN
3,251,2017-01-09,40.235927,36.469868,44.106088,46.256001,45.802666,NaN,NaT,NaN,False,False,1.009912,1018.899549,in,1018.899549,4.0,NaN
4,252,2017-01-10,40.371809,36.251111,44.817703,45.973999,46.010666,45.755999,2017-01-11 20:28:42.573531648,45.962531,True,False,0.993903,1012.687789,out,1018.899549,1.0,18.899549


In [8]:
performance(tsla_backtest, start_value = 1000)

/var/folders/kc/dkd5zshj10bczq4svz0q8v180000gn/T/ipykernel_19184/3368129621.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['benchmarkValue'][i] = data['benchmarkValue'][i-1]*data['percentChange'][i]
/var/folders/kc/dkd5zshj10bczq4svz0q8v180000gn/T/ipykernel_19184/3368129621.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['stratValue'][i] = data['stratValue'][i-1]*data['percentChange'][i]
/var/folders/kc/dkd5zshj10bczq4svz0q8v180000gn/T/ipykernel_19184/3368129621.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: h

In [20]:
tsla_backtest.head()

,indexValue,ds,yhat,yhat_lower,yhat_upper,y,maFast,maSlow,crossDate,crossY,percentChange,benchmarkValue,position,sell,buy,stratValue,grp,profit
0,253,2017-01-04,38.591362,34.443162,43.111233,45.397999,NaN,NaN,NaT,NaN,NaN,1000.000000,in,False,False,1000.000000,1.0,NaN
1,253,2017-01-05,38.896702,34.625595,43.032766,45.349998,NaN,NaN,NaT,NaN,0.998943,998.942677,in,False,False,998.942677,2.0,NaN
2,253,2017-01-06,39.377669,35.202265,43.426187,45.801998,45.516665,NaN,NaT,NaN,1.009967,1008.899056,in,False,False,1008.899056,3.0,NaN
3,251,2017-01-09,40.235927,36.469868,44.106088,46.256001,45.802666,NaN,NaT,NaN,1.009912,1018.899549,in,False,False,1018.899549,4.0,NaN
4,252,2017-01-10,40.371809,36.251111,44.817703,45.973999,46.010666,45.755999,2017-01-11 20:28:42.573531648,45.962531,0.993903,1012.687789,out,True,False,1018.899549,1.0,18.899549


In [21]:
print(tsla_backtest[['stratValue', 'benchmarkValue']])

       stratValue  benchmarkValue
0     1000.000000     1000.000000
1      998.942677      998.942677
2     1008.899056     1008.899056
3     1018.899549     1018.899549
4     1018.899549     1012.687789
...           ...             ...
1271  8787.902388    20791.665914
1272  8658.490317    20485.484479
1273  8550.492170    20229.967146
1274  8727.478683    20648.706941
1275  7719.090441    18262.919012

[1276 rows x 2 columns]


In [49]:
tsla_backtest

,indexValue,ds,yhat,yhat_lower,yhat_upper,y
0,253,2017-01-04,38.591362,34.443162,43.111233,45.397999
1,253,2017-01-05,38.896702,34.625595,43.032766,45.349998
2,253,2017-01-06,39.377669,35.202265,43.426187,45.801998
3,251,2017-01-09,40.235927,36.469868,44.106088,46.256001
4,252,2017-01-10,40.371809,36.251111,44.817703,45.973999
...,...,...,...,...,...,...
1271,253,2022-01-21,1068.214802,986.332349,1142.882445,943.900024
1272,252,2022-01-24,1075.157468,996.914812,1156.858364,930.000000
1273,253,2022-01-25,1061.997755,983.298469,1137.909923,918.400024
1274,253,2022-01-26,1049.307293,972.551491,1126.558127,937.409973
